# Исследование надежности заемщиков.

Задача нашего исследования - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок

Результаты нашего исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Шаги работы:**

- Обзор данных
- Заполнение пропусков
- Проверка данных на анамалии и исправление
- Изменение типов данных
- Удаление дубликатов 
- Декомпозиция исходного датафрейма
- Категоризация дохода
- Категоризация целей кредита 
- Ответы на вопросы 
- Вывод

## Обзор данных

In [1]:
import pandas as pd #импортируем библиотеку

In [2]:
data = pd.read_csv('/Users/alekseivlasov/Desktop/project/2/data.csv')

In [3]:
data.head(10) #открываем первые 10 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info() #получаем общую информацию по датасету

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Название столбцов соответствует хорошему стилю

## Заполнение пропусков

In [5]:
data.isna().sum() #проверим датасет на пропуски

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Найдено 2 столбца с пропусками: `days_employed` и `total_income`

В столбце `total_income`  пропуски `NaN`

In [6]:
pass_income = data.isna().sum()
all_income = data.count()

(pass_income / all_income) * 100

children             0.000000
days_employed       11.234562
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        11.234562
purpose              0.000000
dtype: float64

Доля пропущенных значений 11%

Возможная причина появляение пропусков `total_income` отсутствия опыта, так как количество пропусков одинаково 

Если мы заполним пропуски средним значением или 0, мы можем получить при дальнейших расчетах некорректные выводы, поэтому лучше заменить пропуски медианным значением

Для каждого типа занятости определим медианное значение total_income и заполним пропуски с тем же типом

In [7]:
income_type_list = [
    'сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель', 'безработный', 'в декрете', 'студент']

for item in income_type_list:
    type_median = data.loc[data['income_type'] == item,'total_income'].median()
    data.loc[data['income_type'] == item,'total_income'] = data.loc[data['income_type'] == item,'total_income'].fillna(type_median)

## Проверка данных на аномалии и исправления.

Когда мы знакомились с датасетом, сразу бросились в глаза отрицательные значения в столбце `days_employed` 

In [8]:
data[data['days_employed'] < 0]['days_employed'].count() #проверим количество отрицательных значений

15906

In [9]:
data['days_employed'].max() #посмтрим, какой максимальный трудовой стаж в днях

401755.40047533

In [10]:
max_employed = data['days_employed'].max()
max_employed = max_employed/ 360
max_employed

1115.9872235425833

1115 лет отличный стаж, но не совсем реальный для нашего мира. Скорей всего, стаж заполнился не в днях, а в часах

In [11]:
max_employed/24

46.49946764760764

46 лет - этот стаж больше подходит для нашей планеты 

В столбце `days_employed` найдены отрицательные и завышенные значения:  
Отрицательные значения могли появится при технической ошибке  
Завышенные значения имеют формат не в днях, а в часах.

In [12]:
#для начала мы заменим отрицательные значения положительными 
data['days_employed'] = data['days_employed'].abs()

In [13]:
data[data['days_employed'] < 0]['days_employed'].count()

0

Мы убрали минус и оставили такие же значения, так как не найдено законномерности, почему возникли отрицательные значения

In [14]:
#возьмем за основу стаж 47 лет - 16920 дней, все что выше, будем переводить 
data.loc[data['days_employed'] > 16920, 'days_employed'] = data['days_employed']/24
data['days_employed'].max() #проверим, получилось ли убрать завышенные значения        

16739.80835313875

Все значения, которые были больше 47 лет (16920 дня), мы перевели из часов в дни. Теперь максимальный трудовой стаж выглядит реалистично

В столбце `days_employed` остались пропуски, их мы заполним медианными значениями по типу занятости

In [15]:
for item in income_type_list:
    type_median = data.loc[data['income_type'] == item, 'days_employed'].median()
    data.loc[data['income_type'] == item, 'days_employed'] = data.loc[data['income_type'] == item, 'days_employed'].fillna(type_median)

In [16]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Мы избавились от пропусков в датасете

Нужно проверить столбец, который понадобится для исследования

In [17]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 детей может быть, но смущает, что такое количество детей встречается чаще чем 4 и 5 ребенка, что не похоже на правду. И у нас в списке пристутствует отрицательное значение, такого тоже быть не может  
Так как доля аномалий мала и ни на что не повлияет - мы удалим строчки с аномальными значениями

In [18]:
data = data.loc[data['children'] != 20]
data = data.loc[data['children'] != -1]
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [19]:
data['gender'].value_counts() #проверяем gender на аномалии

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Мы нашли странное значение в данных пола. Так как встречается оно всего один раз - мы его удалим

In [20]:
data = data.loc[data['gender'] != 'XNA'] #удалили строку
data['gender'].value_counts()

F    14154
M     7247
Name: gender, dtype: int64

Нашли аномалии в датасете и обработали их, переходим к следующему этапу

## Изменение типов данных.

In [21]:
data['total_income'] = data['total_income'].astype('int') #изменим вещественный тип данных на целочисленный
data['days_employed'] = data['days_employed'].astype('float32') #для экономии памяти float64 заменим на float32

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21401 non-null  int64  
 1   days_employed     21401 non-null  float32
 2   dob_years         21401 non-null  int64  
 3   education         21401 non-null  object 
 4   education_id      21401 non-null  int64  
 5   family_status     21401 non-null  object 
 6   family_status_id  21401 non-null  int64  
 7   gender            21401 non-null  object 
 8   income_type       21401 non-null  object 
 9   debt              21401 non-null  int64  
 10  total_income      21401 non-null  int64  
 11  purpose           21401 non-null  object 
dtypes: float32(1), int64(6), object(5)
memory usage: 2.0+ MB


## Удаление дубликатов.

Вначале найдем неявные дубликаты, с помощью метода `value_counts()`

In [23]:
data['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      664
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Найдены одинковые значения, но записанные по-разному: с использованием заглавных и строчных букв

In [24]:
data['education'] = data['education'].str.lower() #с помощью метода str.lower приведем к одному регистру
data['education']

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21401, dtype: object

In [25]:
data['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      740
начальное                282
ученая степень             6
Name: education, dtype: int64

In [26]:
data['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

Перебрав столбцы методом `value_counts()` нашли неявные дубликаты в столбце `purpose`. Связаны они скорей всего тем, что столбец заполняется в свободной форме и не привязан к определенным категориям. Его трогать не будем, так как это не цель нашего исследования 

Методом `duplicated()` проверим наличие явных дубликатов, при нахождении удалим

In [27]:
data.duplicated().sum() #проверим наличие явных дубликатов в датасете

71

In [28]:
data = data.drop_duplicates().reset_index(drop=True) #удаляем явные дубликаты

In [29]:
data.duplicated().sum()

0

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим первый датафрейм, где укажем уникальные значения `education` и `education_id`

In [30]:
education_status = data[['education', 'education_id']]
education_status.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
5,высшее,0
6,высшее,0
7,среднее,1
8,высшее,0
9,среднее,1


In [31]:
education_status = education_status.drop_duplicates().reset_index(drop=True)
education_status.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Тоже самое проделываем со вторым датафреймом, который будет состоять из уникальных значений `family_status` и `family_status_id`

In [32]:
family_status = data[['family_status', 'family_status_id']]
family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
5,гражданский брак,1
6,женат / замужем,0
7,женат / замужем,0
8,гражданский брак,1
9,женат / замужем,0


In [33]:
family_status = family_status.drop_duplicates().reset_index(drop=True)
family_status.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [34]:
data = data.drop(columns=['education', 'family_status'])

In [35]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

Удалили из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`

## Категоризация дохода.

In [36]:
#создадим функцию, которая будет присваивать категорию дохода
def total_income_f(total):
    if total <= 30000:
        return 'E'
    if total <= 50000:
        return 'D'
    if total <= 200000:
        return 'C'
    if total <= 1000000:
        return 'B'
    if total > 1000000:
        return 'A'
    else:
        return 'Не подходит под категорию'

In [37]:
total_test = [10, 200000, 50000, 1000000, 30000] #проверим работу нашей функции
for _ in total_test:
    print(total_income_f(_))

E
C
D
B
E


In [38]:
#создали новый столбец и применили функцию методом apply
data['total_income_category'] = data['total_income'].apply(total_income_f)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.672852,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803711,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422852,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747070,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.752930,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


## Категоризация целей кредита.

Посмотрим уникальные значения по цели кредита

In [39]:
data['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      615
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

In [40]:
#создадим функцию, которая будет формировать категории
def purpose_function(purpose):
    try:
        if 'автомобил' in purpose:
            return 'операции с автомобилем'
        if 'недвижим' in purpose or 'жиль' in purpose:
            return 'операции с недвижимостью'
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'образован' in purpose:
            return 'получение образования'
        else:
            return 'такой категории нет'
    except:
        return 'Ошибка'

In [41]:
purpose_test = ['хочу купить автомобиль', 'очень сильно нужно жилье', 'планирую свадьбу', 'доп образование']
for _ in purpose_test:
    print(purpose_function(_)) #проверим функцию

операции с автомобилем
операции с недвижимостью
проведение свадьбы
получение образования


Функция работает

In [42]:
#применяем функцию к датафрейму, с помощью метода apply
data['purpose_category'] = data['purpose'].apply(purpose_function)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.672852,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803711,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422852,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747070,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.752930,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Теперь в датафрейме есть распределение по категориям на цель получения кредита

### Ответы на вопросы.

##### Вопрос 1:

Давайте ответим на 1 вопрос: есть ли зависимость между количеством детей и возвратом кредита в срок?  
Для того, чтобы дать ответ воспользуемся `pivot_table`

In [43]:
children_pivot1 = data.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count', fill_value=0)
children_pivot1['addiction'] = children_pivot1[1]/children_pivot1[0]
children_pivot1

debt,0,1,addiction
children,,,
0,13027,1063,0.081600
1,4364,444,0.101742
2,1858,194,0.104413
3,303,27,0.089109
4,37,4,0.108108
5,9,0,0.000000


In [44]:
children_pivot2 = data.pivot_table(index="children", values="debt", aggfunc=['sum','count','mean'])
children_pivot2

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14090,0.075444
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


##### Вывод 1:

Как показывает сводная таблица - зависимости между количеством детей и возвратом кредита в срок нет, доля отличается на незначительное количество пунктов, каких-то всплесков не обнаружили. Радует, что большая часть возвращает кредит в срок, а те у кого 5 детей, все вовремя платят

##### Вопрос 2:

Давайте также проверим, есть ли зависимость между семейным положением и возвратом кредита в срок

In [45]:
data_family_status = data.merge(family_status, on='family_status_id', how='left') #методом merge склеим 2 таблицы
data_family_status.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,family_status
0,1,8437.672852,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,женат / замужем
1,1,4024.803711,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,женат / замужем
2,0,5623.422852,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,женат / замужем
3,3,4124.747070,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,женат / замужем
4,0,14177.752930,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,гражданский брак


In [46]:
family_pivot = data_family_status.pivot_table(
    index='family_status', columns='debt', values='dob_years', aggfunc='count'
)
family_pivot['addiction'] = family_pivot[1]/family_pivot[0]
family_pivot

debt,0,1,addiction
family_status,,,
Не женат / не замужем,2523,273,0.108205
в разводе,1105,84,0.076018
вдовец / вдова,888,63,0.070946
гражданский брак,3748,385,0.102721
женат / замужем,11334,927,0.081789


##### Вывод 2:

Зависимость между семейным положением и оплатой кредита в срок тоже не обнаружено 

##### Вопрос 3:

Сейчас узнаем, есть ли зависимость между уровнем дохода и возвратом кредита в срок  
Для анализа возьмем данные по категориям дохода

In [47]:
income_pivot = data.pivot_table(index='total_income_category', columns='debt', values='dob_years', aggfunc='count')
income_pivot['addiction'] = income_pivot[1]/income_pivot[0]
income_pivot

debt,0,1,addiction
total_income_category,,,
A,23,2,0.086957
B,4659,354,0.075982
C,14568,1353,0.092875
D,328,21,0.064024
E,20,2,0.100000


##### Вывод 3:

Между уровнем дохода и возвратом кредита зависимости не обнаружили. В сводной таблице хорошо видно, что категории `B` и `C` чаще всего берут кредит

##### Вопрос 4:

В последнем вопросе мы узнаем, как разные цели кредита влияют на его возврат в срок

In [48]:
purpose_pivot = data.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')
purpose_pivot['addiction'] = purpose_pivot[1]/purpose_pivot[0]
purpose_pivot

debt,0,1,addiction
purpose_category,,,
операции с автомобилем,3879,400,0.103119
операции с недвижимостью,9970,780,0.078235
получение образования,3619,369,0.101962
проведение свадьбы,2130,183,0.085915


##### Вывод 4:

От цели кредита не зависит, выплатит клиент в срок или нет

## Общий вывод:

В данном исследовании мы проделали следующие шаги:

**Обзор данных**  
Познакомились с таблицей, просмотрели, какие есть столбцы и какой тип значений в них. Просматривая данные были выявленны пропуски в таблицах

**Заполнение пропусков**  
Пропуски нашли в двух столбцах ``days_employed`` и ``total_income``с одинаковым количеством. Также вычислили долю пропусков, она составила 11%, было принятно решение их не удалять, а заполнить медианными значениями по типу клиента

**Проверка данных на аномалии**  
Проверяя данные на аномалии нашли отрицательные и завышенные значения в общем стаже работы. Отрицательные значения исправили, а завышенные перевели, так как они были не в днях, а в часах  
*Рекомендуем заполнять данных столбец в одном формате "дни"*

В столбце ``children`` аномальными значениями были: "20" и "-1" - мы их удалили, так как доля была незначительная, по сравнению со всем датасетом

Лишнее значение было в столбце ``gender``, значение удалили

**Изменение типов данных**  
Далее изменили типы данных, для дальнейшего исследования

**Удаление дубликатов**
Нашли явные и неявные дубликаты, после чего удалили или привели к единому регистру
*Рекомедуем прописывать значения в одном регистре, чтобы не было дублей

**Фомрирование дополнительных датафреймов**  
Мы создали датафреймы: ``education_status`` и ``education_status`` для того, чтобы упростить работу в основном датасете

**Категоризация**  
Также мы сделали категории по доходу и категории целей кредита, теперь можно анализировать датафрейм по категорям.

---

Задача нашего исследования была - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок

Мы разобрали 4 сводные таблицы по разным типам и все они показывают похожие результаты, нет явных выбросов, разница в 2-3%. А это говорит о том, что количество детей и семейное положение не влияют на возврат кредита вовремя

Клиенты, у которых нет детей больше всех берут кредиты и процент просрочек один из самых маленьких!  
На эту группу стоит обратить внимание

Также, больше всего берут кредиты люди, которые находятся в браке, на втором месте - гражданский брак

Можно обратить внимание на категорию дохода "B" и "С" эти сегменты по количеству привышают остальных, но по проценту возврата также не сильно отличаются